## 보스턴 집값 예측하기

#### 집값의 변동에 여러 영향을 미치는 요인을 모아서 선형 회귀를 테스트해보는 예시이다

|속성|설명|속성|설명|
|:-:|:--|:-:|:--|
|CRIM|인구 1 인당 범죄 발생수|ZIN|20,000 평방 피트 이상의 주거 구역 비중|
|INDUS|소매업 외 상업이 차지하는 면적 비율|CHAS|찰스 강 위치 변수(1:주변, 0:이외)|
|NOX|일산화 질소 농도|RM|집의 평균 방수|
|AGE|1940 년 이전에 지어진 비율 |DIS|5 가지 보스턴 시 고용 시설까지의 거리|
|RAD|순환 고속 도로의 접근 용이성|TAX|`$10,000`당 부동산 세율 총계|
|PTRAITO|지역별 학생과 교사 비율|B|지역별 흑인 비율|
|LSTAT|급여가 낮은 직업에 종사하는 인구 비율| |가격(단위 : $1,000)|

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

df = pd.read_csv('../data/housing.csv', header=None, delim_whitespace=True)
# print(type(df))
# print(type(df.values))  # 데티어 frame을  numpy로 바꾸어야 keras에서 돌아감

In [3]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
5,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222.0,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311.0,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311.0,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311.0,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311.0,15.2,386.71,17.10,18.9


In [4]:
type(df.values) # 데티어 frame을  numpy로 바꾸어야 keras에서 돌아감

numpy.ndarray

In [5]:
dataset = df.values
x = dataset[:, 0:13]
y = dataset[:, 13]

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=seed)

In [7]:
model = Sequential()
model.add(Dense(units=30, input_dim=13, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=200, batch_size=10)

Epoch 1/200
354/354 [==============================] - 1s 1ms/step - loss: 2189.3962
Epoch 2/200
354/354 [==============================] - 0s 251us/step - loss: 115.0912
Epoch 3/200
354/354 [==============================] - 0s 268us/step - loss: 77.4182
Epoch 4/200
354/354 [==============================] - 0s 268us/step - loss: 71.3885
Epoch 5/200
354/354 [==============================] - 0s 281us/step - loss: 69.7504
Epoch 6/200
354/354 [==============================] - 0s 270us/step - loss: 67.8975
Epoch 7/200
354/354 [==============================] - 0s 400us/step - loss: 63.8973
Epoch 8/200
354/354 [==============================] - 0s 248us/step - loss: 62.8484
Epoch 9/200
354/354 [==============================] - 0s 284us/step - loss: 61.0540
Epoch 10/200
354/354 [==============================] - 0s 234us/step - loss: 60.1971
Epoch 11/200
354/354 [==============================] - 0s 243us/step - loss: 57.8535
Epoch 12/200
354/354 [==============================] - 0s 259

In [8]:
# 예측 값과 실제 값의 비교
Y_prediction = model.predict(x_test).flatten()

In [9]:
for i in range(10):
    label = y_test[i]
    prediction = Y_prediction[i]
    print(("실제 가격: {:.3f}, 예상가격: {:.3f}".format(label, prediction)))

실제 가격: 22.600, 예상가격: 20.359
실제 가격: 50.000, 예상가격: 26.190
실제 가격: 23.000, 예상가격: 21.996
실제 가격: 8.300, 예상가격: 12.359
실제 가격: 21.200, 예상가격: 18.423
실제 가격: 19.900, 예상가격: 21.970
실제 가격: 20.600, 예상가격: 19.250
실제 가격: 18.700, 예상가격: 24.139
실제 가격: 16.100, 예상가격: 18.991
실제 가격: 18.600, 예상가격: 13.473
